# LLM LangChain : Model, Prompt and Output

In [2]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
import os
import openai

openai.api_base = "http://localhost:12346/v1"
openai.api_key = "EMPTY"

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.7,  
    )
    return response.choices[0].message["content"]

get_completion("What is 1+1?", model="MPT-7B-Instruct")

'1 + 1 = 2'

In [4]:
from langchain.chat_models import ChatOpenAI

# Model Creation

chat = ChatOpenAI(temperature=0.0, model='MPT-7B-Instruct', openai_api_key="EMPTY", openai_api_base="http://localhost:12346/v1")
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='MPT-7B-Instruct', temperature=0.0, model_kwargs={}, openai_api_key='EMPTY', openai_api_base='http://localhost:12346/v1', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

In [6]:
# Defining Prompt template

from langchain.prompts import ChatPromptTemplate

template_string = """Write the code in {language} for the text which is delimmited \
by triple backticks\
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

print  (prompt_template.messages[0].prompt, prompt_template.input_variables)

# formating message using template
prompt_message = prompt_template.format_messages(language='Python', text='the sum of n numbers' )

print ('prompt_message: ', prompt_message)

# calling LLM 

response = chat(prompt_message)

print('response: ', response)

input_variables=['language', 'text'] output_parser=None partial_variables={} template='Write the code in {language} for the text which is delimmited by triple backtickstext: ```{text}```\n' template_format='f-string' validate_template=True ['language', 'text']
prompt_message:  [HumanMessage(content='Write the code in Python for the text which is delimmited by triple backtickstext: ```the sum of n numbers```\n', additional_kwargs={}, example=False)]
response:  content="```n = int(input('Enter a number: '))\nsum = 0\nfor i in range(n):\n    sum += i\nprint(sum)```" additional_kwargs={} example=False


In [15]:
# Extarcting information from text
from langchain.prompts import ChatPromptTemplate

template_string = """Extract the information from the text which is delimmited \
by triple backticks\

Person: names of the persons otherwise output ""
Meeting Place: the place of meeting otherwise output ""
Day : day of the meeting otherwise output ""
Date: date of the meeting expmple Monday, Tuesday ... otherwise output ""

convert extracted information into JSON format using keys
Person
Meeting Place
Date
Day

text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

print  (prompt_template.messages[0].prompt, prompt_template.input_variables)

# formating message using template
prompt_message = prompt_template.format_messages(text='Ram meets Laxman on on 10th January 2013 in Delhi, India' )

print ('prompt_message: ', prompt_message)

# calling LLM 

response = chat(prompt_message)

print (response.content, type(response.content))


# output is str convert it into dictionary
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser




input_variables=['text'] output_parser=None partial_variables={} template='Extract the information from the text which is delimmited by triple backticks\nPerson: names of the persons otherwise output ""\nMeeting Place: the place of meeting otherwise output ""\nDay : day of the meeting otherwise output ""\nDate: date of the meeting expmple Monday, Tuesday ... otherwise output ""\n\nconvert extracted information into JSON format using keys\nPerson\nMeeting Place\nDate\nDay\n\ntext: ```{text}```\n' template_format='f-string' validate_template=True ['text']
prompt_message:  [HumanMessage(content='Extract the information from the text which is delimmited by triple backticks\nPerson: names of the persons otherwise output ""\nMeeting Place: the place of meeting otherwise output ""\nDay : day of the meeting otherwise output ""\nDate: date of the meeting expmple Monday, Tuesday ... otherwise output ""\n\nconvert extracted information into JSON format using keys\nPerson\nMeeting Place\nDate\nDay